In [612]:
from collections import namedtuple, defaultdict
import csv
from itertools import groupby
import pandas as pd
from scipy.optimize import minimize

column_names = ['portfolio_id', 'portfolio_name', 'name', 'id', 'location', 'fuel', 'capacity', 'fixed_cost', 'emission_cost', 'var_cost']
Generator = namedtuple('Generator', column_names)
Demand = namedtuple('Demand', ['intercept', 'slope'])
stdevs = [236.81, 307.29, 408.86, 351.36]

In [613]:
df = pd.read_csv("gens.csv", names=column_names)
df.head()

,portfolio_id,portfolio_name,name,id,location,fuel,capacity,fixed_cost,emission_cost,var_cost
0,1,Big Coal,FOUR_CORNERS,11,South,Coal,1900,8000,33.0,36.50
1,1,Big Coal,ALAMITOS_7,12,South,Gas,250,0,25.5,73.72
2,1,Big Coal,HUNTINGTON_BEACH_1&2,13,South,Gas,300,2000,13.8,40.50
3,1,Big Coal,HUNTINGTON_BEACH_5,14,South,Gas,150,2000,23.1,66.50
4,1,Big Coal,REDONDO_5&6,15,South,Gas,350,3000,14.4,41.94


In [614]:
demands = []
with open('demands.csv', newline='') as csvfile:
    for demand in csv.reader(csvfile):
        demands.append(Demand(int(demand[0]), float(demand[1])))

In [615]:
generators = {}
with open('gens.csv', newline='', encoding='utf-8-sig') as csvfile:
    for generator in csv.reader(csvfile):
        generator[0] = int(generator[0])
        generator[3] = int(generator[3])
        generator[6] = int(generator[6])
        generator[7] = int(generator[7])
        generator[8] = float(generator[8])
        generator[9] = float(generator[9])
        generators[generator[3]] = Generator(*generator)

In [616]:
def clearing_price(bids, intercept, slope):
    """
    bids=[{price, capacity, id}]
    """
    bids.sort(key=lambda x: x['price'])
    total_capacity = 0
    cleared_gens = defaultdict(int)
    for k, g in groupby(bids, key=lambda x: x['price']):
        generators = list(g)
        sum_capacity = sum(map(lambda x: x['capacity'], generators))
        total_capacity += sum_capacity
        clearing_price = k
        q_total = intercept + slope * k
        if total_capacity < q_total:
            for bid in generators:
                cleared_gens[bid['id']] = bid['capacity']
        else:
            for bid in generators:
                cleared_gens[bid['id']] = (sum_capacity - (total_capacity - q_total)) * bid['capacity'] / sum_capacity
            break
    

    return clearing_price, cleared_gens
            
            
            

In [617]:
clearing_price([{"price": 100, "capacity": 200, "id": 1}, {"price": 80, "capacity": 200, "id": 2}, {"price": 120, "capacity": 200, "id": 3}, {"price": 100, "capacity": 200, "id": 4}], 1000, -3)

(120, defaultdict(int, {2: 200, 1: 200, 4: 200, 3: 40.0}))

In [618]:
def profit(price, var_cost, quantity):
    return (price - var_cost) * quantity

In [619]:
def total_profit(generators, ids, bids, intercept, slope):
    bids_array = [{"price": v, "capacity": generators[k].capacity, "id": k} for k, v in bids.items()]
    p, clearing_gens = clearing_price(bids_array, intercept, slope)
    total_profit = 0
    for _id in ids:
        price = bids[_id]
        var_cost = generators[_id].var_cost
        total_profit += profit(price, var_cost, clearing_gens.get(_id, 0))
        
    return total_profit

In [620]:
# everybody bids at marginal cost

bids = {_id: generators[_id].var_cost for _id in generators}

In [621]:
# Portfolio 7
ids = [71, 72, 73, 74, 75, 76, 77, 78]

def profit_p7(initial_bids, intercept, slope):
    for idx, _id in enumerate(ids):
        bids[_id] = initial_bids[idx]
    return total_profit(generators, ids, bids, intercept, slope)

In [622]:
initial_bids = [generators[_id].var_cost+0.02 for _id in ids]

for i in range(4):
    intercept = demands[i].intercept
    slope = demands[i].slope
    _profit = lambda x: -profit_p7(x, intercept, slope)
    res = minimize(_profit, initial_bids, method='nelder-mead', options={'xatol': 1e-8, 'disp': True, 'maxiter': 50000,  'fatol': 0.0000001})
    c_price, cleared_gens = clearing_price([{"price": price, "id": index, "capacity": generators[index].capacity} for index, price in bids.items()], intercept, slope)
    print(f"d1h{i+1}", c_price)
    df[f"bids_d1h{i+1}"] = df['id'].map(bids)
    df[f"quantities_d1h{i+1}"] = df['id'].map(cleared_gens)
    df[f"profits_d1h{i+1}"] = (df[f'bids_d1h{i+1}'] - df['var_cost']) * df[f"quantities_d1h{i+1}"]

Optimization terminated successfully.
         Current function value: -1973.332800
         Iterations: 574
         Function evaluations: 1076
d1h1 39.499999999999986
Optimization terminated successfully.
         Current function value: -5837.000000
         Iterations: 669
         Function evaluations: 1190
d1h2 41.67
Optimization terminated successfully.
         Current function value: -29364.228272
         Iterations: 2683
         Function evaluations: 4557
d1h3 52.496988201395965
Optimization terminated successfully.
         Current function value: -9407.931332
         Iterations: 629
         Function evaluations: 1142
d1h4 43.829999999998556


In [623]:
df[['id','name','var_cost','capacity', 'bids_d1h3', 'quantities_d1h3', 'profits_d1h3']].tail(8)

,id,name,var_cost,capacity,bids_d1h3,quantities_d1h3,profits_d1h3
34,71,COOLWATER,42.39,650,55.312108,0.000000,0.000000
35,72,ETIWANDA_1-4,42.67,850,52.496988,850.000000,8352.939971
36,73,ETIWANDA_5,62.89,150,53.616699,0.000000,-0.000000
37,74,ELLWOOD,75.61,300,55.402521,0.000000,-0.000000
38,75,MANDALAY_1&2,39.06,300,41.667313,300.000000,782.193966
39,76,MANDALAY_3,52.06,150,52.496988,42.292137,18.481165
40,77,ORMOND_BEACH_1,38.06,700,52.495316,700.000000,10104.721429
41,78,ORMOND_BEACH_2,38.06,700,52.496988,700.000000,10105.891741


In [624]:
bids[71] = 52.47
bids[72] = 52.48
bids[73] = 500
bids[74] = 500
bids[75] = 52.46
bids[76] = 52.49
bids[77] = 52.45
bids[78] = 52.45

In [625]:
c_price, cleared_gens = clearing_price([{"price": price, "id": index, "capacity": generators[index].capacity} for index, price in bids.items()], demands[2].intercept, demands[2].slope)

In [626]:
c_price

52.48

In [627]:
cleared_gens

defaultdict(int,
            {21: 1000,
             42: 800,
             45: 1000,
             63: 750,
             64: 750,
             22: 750,
             23: 750,
             11: 1900,
             32: 650,
             62: 665,
             61: 335,
             35: 700,
             13: 300,
             31: 650,
             52: 650,
             55: 950,
             15: 350,
             16: 950,
             57: 700,
             51: 400,
             41: 150,
             43: 150,
             24: 150,
             77: 700,
             78: 700,
             75: 300,
             71: 650,
             72: 242.38880000000063})

In [628]:
df[f"bids_d1h3_"] = df['id'].map(bids)
df[f"quantities_d1h3_"] = df['id'].map(cleared_gens)
df[f"profits_d1h3_"] = (df[f'bids_d1h3_'] - df['var_cost']) * df[f"quantities_d1h3_"]

In [629]:
df[['id','bids_d1h1', 'quantities_d1h1', 'profits_d1h1', 'bids_d1h2', 'quantities_d1h2', 'profits_d1h2', 'bids_d1h3_', 'quantities_d1h3_', 'profits_d1h3_', 'bids_d1h4', 'quantities_d1h4', 'profits_d1h4']].tail(8)

,id,bids_d1h1,quantities_d1h1,profits_d1h1,bids_d1h2,quantities_d1h2,profits_d1h2,bids_d1h3_,quantities_d1h3_,profits_d1h3_,bids_d1h4,quantities_d1h4,profits_d1h4
34,71,43.120682,0.000,0.0000,42.862691,0.0,0.0,52.47,650.0000,6552.000000,44.910439,0.0000,0.000000
35,72,40.402647,0.000,-0.0000,42.505169,0.0,-0.0,52.48,242.3888,2377.834128,45.355823,0.0000,0.000000
36,73,62.529961,0.000,-0.0000,60.520050,0.0,-0.0,500.00,0.0000,0.000000,66.595883,0.0000,0.000000
37,74,75.678316,0.000,0.0000,73.991389,0.0,-0.0,500.00,0.0000,0.000000,63.763119,0.0000,-0.000000
38,75,39.706341,0.000,0.0000,41.670000,300.0,783.0,52.46,300.0000,4020.000000,43.830000,278.8116,1329.931332
39,76,52.772872,0.000,0.0000,48.641896,0.0,-0.0,52.49,0.0000,0.000000,46.520096,0.0000,-0.000000
40,77,39.500000,685.185,986.6664,41.670000,700.0,2527.0,52.45,700.0000,10073.000000,43.830000,700.0000,4039.000000
41,78,39.500000,685.185,986.6664,41.670000,700.0,2527.0,52.45,700.0000,10073.000000,43.830000,700.0000,4039.000000


In [630]:
sum(df['profits_d1h3'])

29364.228272013366

In [631]:
sum(df['profits_d1h3_'])

33095.834128

In [632]:
df

,portfolio_id,portfolio_name,name,id,location,fuel,capacity,fixed_cost,emission_cost,var_cost,...,profits_d1h2,bids_d1h3,quantities_d1h3,profits_d1h3,bids_d1h4,quantities_d1h4,profits_d1h4,bids_d1h3_,quantities_d1h3_,profits_d1h3_
0,1,Big Coal,FOUR_CORNERS,11,South,Coal,1900,8000,33.0,36.50,...,0.0,36.500000,1900.000000,0.000000,36.500000,1900.0000,0.000000,36.50,1900.0000,0.000000
1,1,Big Coal,ALAMITOS_7,12,South,Gas,250,0,25.5,73.72,...,0.0,73.720000,0.000000,0.000000,73.720000,0.0000,0.000000,73.72,0.0000,0.000000
2,1,Big Coal,HUNTINGTON_BEACH_1&2,13,South,Gas,300,2000,13.8,40.50,...,0.0,40.500000,300.000000,0.000000,40.500000,300.0000,0.000000,40.50,300.0000,0.000000
3,1,Big Coal,HUNTINGTON_BEACH_5,14,South,Gas,150,2000,23.1,66.50,...,0.0,66.500000,0.000000,0.000000,66.500000,0.0000,0.000000,66.50,0.0000,0.000000
4,1,Big Coal,REDONDO_5&6,15,South,Gas,350,3000,14.4,41.94,...,0.0,41.940000,350.000000,0.000000,41.940000,350.0000,0.000000,41.94,350.0000,0.000000
5,1,Big Coal,REDONDO_7&8,16,South,Gas,950,5000,14.4,41.94,...,0.0,41.940000,950.000000,0.000000,41.940000,950.0000,0.000000,41.94,950.0000,0.000000
6,2,Old Timers,BIG_CREEK,21,South,Hydro,1000,15000,0.0,0.00,...,0.0,0.000000,1000.000000,0.000000,0.000000,1000.0000,0.000000,0.00,1000.0000,0.000000
7,2,Old Timers,MOHAVE_1,22,South,Coal,750,15000,28.2,34.50,...,0.0,34.500000,750.000000,0.000000,34.500000,750.0000,0.000000,34.50,750.0000,0.000000
8,2,Old Timers,MOHAVE_2,23,South,Coal,750,15000,28.2,34.50,...,0.0,34.500000,750.000000,0.000000,34.500000,750.0000,0.000000,34.50,750.0000,0.000000
9,2,Old Timers,HIGHGROVE,24,South,Gas,150,0,17.4,49.61,...,0.0,49.610000,150.000000,0.000000,49.610000,0.0000,0.000000,49.61,150.0000,0.000000
